In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from datetime import datetime

In [67]:
%run Data_Cleaning_Utils.ipynb

In [68]:
data_path = './PM2.5/'
year_scope = [2017, 2018, 2019, 2020, 2021, 2022]
os.listdir(data_path)
data_name_lst = [name for name in os.listdir(data_path) if name.startswith('PM2.5')]

In [69]:
selected_data_name = []
for i in year_scope:
    for j in data_name_lst:
        if str(i) in j:
            selected_data_name.append(j)

In [25]:
selected_data_name

['PM2.5_2017_.xlsx',
 'PM2.5_2018_.xlsx',
 'PM2.5_2019_.xlsx',
 'PM2.5_2020_.xlsx',
 'PM2.5_2021_.xlsx',
 'PM2.5_2022_.xlsx']

## Working Space

### Observing on the provide

In [62]:
provide_station_df = made_provide_station(selected_data_name, data_path)

In [64]:
provide_station_df.sort_values(['label'])

,station_id,station_name,label
0,05T,กทม.,0
1,59T,กทม.,0
2,61T,กทม.,0
3,50T,กทม.,0
4,53T,กทม.,0
...,...,...,...
15,36T,จ.เชียงใหม่,51
84,97T,จ.เพชรบูรณ์,52
51,72T,จ.เลย,53
48,69T,จ.แพร่,54


In [65]:
PM_data = load_selected_PM_data(data_path, selected_data_name)

In [57]:
PM_data['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,40T,44T,46T,62T,63T,75T,76T,77T,79T,80T
0,2017-01-01,33.0,21.0,28.0,NaN,34.0,NaN,NaN,28.0,NaN,...,21.0,7.0,NaN,NaN,NaN,16.0,20.0,18.0,NaN,NaN
1,2017-01-02,28.0,19.0,27.0,NaN,32.0,NaN,NaN,26.0,NaN,...,11.0,9.0,NaN,NaN,NaN,5.0,18.0,15.0,NaN,NaN
2,2017-01-03,31.0,19.0,32.0,NaN,34.0,NaN,NaN,28.0,NaN,...,12.0,6.0,NaN,NaN,NaN,4.0,21.0,19.0,NaN,NaN
3,2017-01-04,37.0,21.0,34.0,NaN,37.0,NaN,NaN,28.0,NaN,...,10.0,5.0,37.0,NaN,NaN,2.0,18.0,21.0,NaN,NaN
4,2017-01-05,45.0,29.0,40.0,NaN,45.0,NaN,NaN,34.0,NaN,...,7.0,7.0,42.0,NaN,NaN,NaN,12.0,29.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2017-12-27,38.0,NaN,46.0,45.0,52.0,25.0,NaN,56.0,48.0,...,21.0,9.0,29.0,10.0,16.0,29.0,21.0,27.0,15.0,19.0
361,2017-12-28,18.0,21.0,26.0,29.0,37.0,18.0,NaN,30.0,30.0,...,6.0,8.0,18.0,7.0,17.0,NaN,9.0,23.0,8.0,10.0
362,2017-12-29,14.0,15.0,20.0,24.0,31.0,15.0,NaN,24.0,23.0,...,11.0,9.0,17.0,11.0,15.0,11.0,10.0,25.0,15.0,9.0
363,2017-12-30,14.0,22.0,22.0,24.0,31.0,16.0,NaN,26.0,24.0,...,12.0,9.0,11.0,12.0,17.0,12.0,14.0,25.0,20.0,8.0


In [21]:
def match_provide_station(PM_data_year, provide_station_df):
    provide_label_dict = {'Date':PM_data_year['Date']}
    station_columns = PM_data_year.columns[1:]
    for station_id in station_columns:
        provide_label = provide_station_df['provide_label'][provide_station_df['station_id'] == station_id].to_list()[0]
        if str(provide_label) not in provide_label_dict.keys():
            provide_label_dict[str(provide_label)] = PM_data_year[station_id]
        else:
            provide_label_dict[str(provide_label)] = pd.concat([provide_label_dict[str(provide_label)], PM_data_year[station_id]], axis=1)
    return provide_label_dict

In [22]:
provide_label_dict = match_provide_station(PM_data['2017'], provide_station_df)

In [62]:
def fill_na_a_provide(df):
    if len(df.shape) > 1:
        for station_id in df.columns:
            df_station = df[station_id].copy()
            idx_for_cal = df.columns.to_list().copy()
            idx_for_cal.pop(idx_for_cal.index(station_id))
            fill_value = df[df_station.isna()][idx_for_cal].mean(axis=1)
            full_df_station = df_station.fillna(fill_value)
            df[station_id] = full_df_station
    return df

def fill_na_all_provide(provide_label_dict):
    new_provide_label_dict = {}
    for id, df in provide_label_dict.items():
        new_provide_label_dict[id] = fill_na_a_provide(df)
    return new_provide_label_dict            

In [63]:
new_provide_label_dict = fill_na_all_provide(provide_label_dict.copy())

In [70]:
new_provide_label_dict['0']

,05T,59T,61T,50T,52T,53T,54T
0,33.0,21.0,28.0,29.00,34.0,29.00,29.000000
1,28.0,19.0,27.0,26.50,32.0,26.50,26.500000
2,31.0,19.0,32.0,29.00,34.0,29.00,29.000000
3,37.0,21.0,34.0,32.25,37.0,32.25,32.250000
4,45.0,29.0,40.0,39.75,45.0,39.75,39.750000
...,...,...,...,...,...,...,...
360,38.0,41.2,46.0,45.00,52.0,25.00,41.200000
361,18.0,21.0,26.0,29.00,37.0,18.00,24.833333
362,14.0,15.0,20.0,24.00,31.0,15.00,19.833333
363,14.0,22.0,22.0,24.00,31.0,16.00,21.500000


In [78]:
def convert_to_original_providefill(provide_label_dict):
    new_df = provide_label_dict['Date']
    for name_col in list(provide_label_dict.keys())[1:]:
        new_df = pd.concat([new_df, provide_label_dict[name_col]], axis=1)
    return new_df


In [79]:
new_df = convert_to_original_providefill(provide_label_dict)

In [80]:
new_df

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,40T,44T,46T,62T,63T,75T,76T,77T,79T,80T
0,2017-01-01,33.0,21.0,28.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,21.0,7.0,NaN,NaN,NaN,16.0,20.0,18.0,NaN,NaN
1,2017-01-02,28.0,19.0,27.0,26.50,32.0,26.50,26.500000,26.0,26.0,...,11.0,9.0,NaN,NaN,NaN,5.0,18.0,15.0,NaN,NaN
2,2017-01-03,31.0,19.0,32.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,12.0,6.0,NaN,NaN,NaN,4.0,21.0,19.0,NaN,NaN
3,2017-01-04,37.0,21.0,34.0,32.25,37.0,32.25,32.250000,28.0,28.0,...,10.0,5.0,37.0,NaN,NaN,2.0,18.0,21.0,NaN,NaN
4,2017-01-05,45.0,29.0,40.0,39.75,45.0,39.75,39.750000,34.0,34.0,...,7.0,7.0,42.0,NaN,NaN,NaN,12.0,29.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2017-12-27,38.0,41.2,46.0,45.00,52.0,25.00,41.200000,56.0,48.0,...,21.0,9.0,29.0,10.0,16.0,29.0,21.0,27.0,15.0,19.0
361,2017-12-28,18.0,21.0,26.0,29.00,37.0,18.00,24.833333,30.0,30.0,...,6.0,8.0,18.0,7.0,17.0,NaN,9.0,23.0,8.0,10.0
362,2017-12-29,14.0,15.0,20.0,24.00,31.0,15.00,19.833333,24.0,23.0,...,11.0,9.0,17.0,11.0,15.0,11.0,10.0,25.0,15.0,9.0
363,2017-12-30,14.0,22.0,22.0,24.00,31.0,16.00,21.500000,26.0,24.0,...,12.0,9.0,11.0,12.0,17.0,12.0,14.0,25.0,20.0,8.0


## Finished product

In [70]:
data_path = './PM2.5/'
selected_years = [2017, 2018, 2019, 2020, 2021, 2022]
save_path='./Clean_Data/'

label_df = made_provide_station(selected_data_name, data_path)

In [61]:
fill_nan_provide = Fill_NaN_Label(data_path, selected_years, label_df)
new_PM_data = fill_nan_provide.main()

In [60]:
PM_data['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,40T,44T,46T,62T,63T,75T,76T,77T,79T,80T
0,2017-01-01,33.0,21.0,28.0,NaN,34.0,NaN,NaN,28.0,NaN,...,21.0,7.0,NaN,NaN,NaN,16.0,20.0,18.0,NaN,NaN
1,2017-01-02,28.0,19.0,27.0,NaN,32.0,NaN,NaN,26.0,NaN,...,11.0,9.0,NaN,NaN,NaN,5.0,18.0,15.0,NaN,NaN
2,2017-01-03,31.0,19.0,32.0,NaN,34.0,NaN,NaN,28.0,NaN,...,12.0,6.0,NaN,NaN,NaN,4.0,21.0,19.0,NaN,NaN
3,2017-01-04,37.0,21.0,34.0,NaN,37.0,NaN,NaN,28.0,NaN,...,10.0,5.0,37.0,NaN,NaN,2.0,18.0,21.0,NaN,NaN
4,2017-01-05,45.0,29.0,40.0,NaN,45.0,NaN,NaN,34.0,NaN,...,7.0,7.0,42.0,NaN,NaN,NaN,12.0,29.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2017-12-27,38.0,NaN,46.0,45.0,52.0,25.0,NaN,56.0,48.0,...,21.0,9.0,29.0,10.0,16.0,29.0,21.0,27.0,15.0,19.0
361,2017-12-28,18.0,21.0,26.0,29.0,37.0,18.0,NaN,30.0,30.0,...,6.0,8.0,18.0,7.0,17.0,NaN,9.0,23.0,8.0,10.0
362,2017-12-29,14.0,15.0,20.0,24.0,31.0,15.0,NaN,24.0,23.0,...,11.0,9.0,17.0,11.0,15.0,11.0,10.0,25.0,15.0,9.0
363,2017-12-30,14.0,22.0,22.0,24.0,31.0,16.0,NaN,26.0,24.0,...,12.0,9.0,11.0,12.0,17.0,12.0,14.0,25.0,20.0,8.0


In [59]:
new_PM_data['2017']

,Date,05T,59T,61T,50T,52T,53T,54T,08T,19T,...,40T,44T,46T,62T,63T,75T,76T,77T,79T,80T
0,2017-01-01,33.0,21.0,28.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,21.0,7.0,NaN,NaN,NaN,16.0,20.0,18.0,NaN,NaN
1,2017-01-02,28.0,19.0,27.0,26.50,32.0,26.50,26.500000,26.0,26.0,...,11.0,9.0,NaN,NaN,NaN,5.0,18.0,15.0,NaN,NaN
2,2017-01-03,31.0,19.0,32.0,29.00,34.0,29.00,29.000000,28.0,28.0,...,12.0,6.0,NaN,NaN,NaN,4.0,21.0,19.0,NaN,NaN
3,2017-01-04,37.0,21.0,34.0,32.25,37.0,32.25,32.250000,28.0,28.0,...,10.0,5.0,37.0,NaN,NaN,2.0,18.0,21.0,NaN,NaN
4,2017-01-05,45.0,29.0,40.0,39.75,45.0,39.75,39.750000,34.0,34.0,...,7.0,7.0,42.0,NaN,NaN,NaN,12.0,29.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2017-12-27,38.0,41.2,46.0,45.00,52.0,25.00,41.200000,56.0,48.0,...,21.0,9.0,29.0,10.0,16.0,29.0,21.0,27.0,15.0,19.0
361,2017-12-28,18.0,21.0,26.0,29.00,37.0,18.00,24.833333,30.0,30.0,...,6.0,8.0,18.0,7.0,17.0,NaN,9.0,23.0,8.0,10.0
362,2017-12-29,14.0,15.0,20.0,24.00,31.0,15.00,19.833333,24.0,23.0,...,11.0,9.0,17.0,11.0,15.0,11.0,10.0,25.0,15.0,9.0
363,2017-12-30,14.0,22.0,22.0,24.00,31.0,16.00,21.500000,26.0,24.0,...,12.0,9.0,11.0,12.0,17.0,12.0,14.0,25.0,20.0,8.0
